# Data cleaning
The 2016 US Election dataset for primary results contains 3 data tables:
- Primary_results
- County_facts
- County_facts_dictionary

### Download
Data is able to download : https://www.kaggle.com/datasets/benhamner/2016-us-election/

### Data
- 'primary_data' contains information about state, county, parties and how much each candidate collected votes and fraction size of it.
- 'county_facts' dataset contains demographic information such as area names (counties) and the population distribution within those areas.
- 'country_facts_dictionary' contains explanations of codes in 'County_facts' columns names.

In [1]:
import pandas as pd

In [6]:
import sys, os

path = os.path.normpath(os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir, 'Data_original')))
if (not (path in sys.path)) :
    sys.path.append(path)

script_dir = os.path.dirname('Data_original')
csv_file_path = os.path.join(script_dir, 'Data_original')

In [7]:
primary_results = pd.read_csv(os.path.join(csv_file_path, "primary_results.csv"))
primary_results.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,District of Columbia,DC,District of Columbia,11001.0,Democrat,Hillary Clinton,282830.0,0.9086
1,District of Columbia,DC,District of Columbia,11001.0,Republican,Donald Trump,12723.0,0.0409
2,Minnesota,MN,Aitkin,27001.0,Democrat,Bernie Sanders,1014.0,0.2380
3,Minnesota,MN,Aitkin,27001.0,Democrat,Hillary Clinton,2981.0,0.7020
4,Minnesota,MN,Aitkin,27001.0,Republican,Donald Trump,4500.0,0.6000


In [8]:
country_facts = pd.read_csv(os.path.join(csv_file_path,"county_facts.csv"))
country_facts.head()

,fips,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,0,United States,NaN,318857056,308758105,3.3,308745538,6.2,23.1,14.5,...,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,1046363,3531905.43,87.4
1,1000,Alabama,NaN,4849377,4780127,1.4,4779736,6.1,22.8,15.3,...,1.2,28.1,112858843,52252752,57344851,12364,6426342,13369,50645.33,94.4
2,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,13.8,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
3,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,18.7,...,1.3,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6
4,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,16.5,...,0.0,27.0,0,0,188337,6334,0,8,884.88,31.0


In [9]:
county_data = country_facts

In [10]:
country_facts_dictionary = pd.read_csv(os.path.join(csv_file_path,"county_facts_dictionary.csv"))
country_facts_dictionary.head()

,column_name,description
0,PST045214,"Population, 2014 estimate"
1,PST040210,"Population, 2010 (April 1) estimates base"
2,PST120214,"Population, percent change - April 1, 2010 to ..."
3,POP010210,"Population, 2010"
4,AGE135214,"Persons under 5 years, percent, 2014"


In [32]:
republican_votes = primary_results[primary_results['party'] == 'Republican']
democrat_votes = primary_results[primary_results['party'] == 'Democrat']

# Define custom aggregation function to find the candidate with the most votes
def max_votes_candidate(df: pd.DataFrame):
    """
    Calculating the index of the maximum value in two columns 'votes'
    and 'fraction_votes'. It then creates a Pandas Series
    containing information about the winning candidate.
    """
    max_votes_idx = df['votes'].idxmax()
    max_fraction_votes_idx = df['fraction_votes'].idxmax()
    return pd.Series({'Vote': df.loc[max_fraction_votes_idx, 'fraction_votes'],
                      'votes': df.loc[max_votes_idx, 'votes'],
                      'winner': df.loc[max_votes_idx, 'candidate']})

# Group by state_abbreviation and county and summarize for each party
republican_votes = republican_votes.groupby(['state_abbreviation', 'fips']).apply(max_votes_candidate).reset_index()
democrat_votes = democrat_votes.groupby(['state_abbreviation', 'fips']).apply(max_votes_candidate).reset_index()

Data for futher analysis has been divided into two datasets:
- 'respublican_votes'
- 'democrat_votes'

Some data is missing, for Minnesota and DC Columbia, which been adjusted via Excel.
Sources:
- Minnesota: https://www.politico.com/2016-election/results/map/president/minnesota/
- DC Columbia: https://uselectionatlas.org/RESULTS/state.php?year=2016&fips=11&off=0&elect=0

In [33]:
county_data.rename(columns= {"PST045214": "Population, 2014 estimate",
                        "PST040210": "Population, 2010 (April 1) estimates base",
                        "PST120214": "Population, percent change - April 1, 2010 to July 1, 2014",
                        "POP010210": "Population, 2010",
                        "AGE135214": "Persons under 5 years, percent, 2014",
                        "AGE295214": "Persons under 18 years, percent, 2014",
                        "AGE775214": "Persons 65 years and over, percent, 2014",
                        "SEX255214": "Female persons, percent, 2014",
                        "RHI125214": "White alone, percent, 2014",
                        "RHI225214": "Black or African American alone, percent, 2014",
                        "RHI325214": "American Indian and Alaska Native alone, percent, 2014",
                        "RHI425214": "Asian alone, percent, 2014",
                        "RHI525214": "Native Hawaiian and Other Pacific Islander alone, percent, 2014",
                        "RHI625214": "Two or More Races, percent, 2014",
                        "RHI725214": "Hispanic or Latino, percent, 2014",
                        "RHI825214": "White alone, not Hispanic or Latino, percent, 2014",
                        "POP715213": "Living in same house 1 year & over, percent, 2009-2013",
                        "POP645213": "Foreign born persons, percent, 2009-2013",
                        "POP815213": "Language other than English spoken at home, percent, age 5+, 2009-2013",
                        "EDU635213": "High school graduate or higher, percent of persons age 25+, 2009-2013",
                        "EDU685213": "Bachelor's degree or higher, percent of persons age 25+, 2009-2013",
                        "VET605213": "Veterans, 2009-2013",
                        "LFE305213": "Mean travel time to work (minutes), workers age 16+, 2009-2013",
                        "HSG010214": "Housing units, 2014",
                        "HSG445213": "Homeownership rate, 2009-2013",
                        "HSG096213": "Housing units in multi-unit structures, percent, 2009-2013",
                        "HSG495213": "Median value of owner-occupied housing units, 2009-2013",
                        "HSD410213": "Households, 2009-2013",
                        "HSD310213": "Persons per household, 2009-2013",
                        "INC910213": "Per capita money income in past 12 months (2013 dollars), 2009-2013",
                        "INC110213": "Median household income, 2009-2013",
                        "PVY020213": "Persons below poverty level, percent, 2009-2013",
                        "BZA010213": "Private nonfarm establishments, 2013",
                        "BZA110213": "Private nonfarm employment, 2013",
                        "BZA115213": "Private nonfarm employment, percent change, 2012-2013",
                        "NES010213": "Nonemployer establishments, 2013",
                        "SBO001207": "Total number of firms, 2007",
                        "SBO315207": "Black-owned firms, percent, 2007",
                        "SBO115207": "American Indian- and Alaska Native-owned firms, percent, 2007",
                        "SBO215207": "Asian-owned firms, percent, 2007",
                        "SBO515207": "Native Hawaiian- and Other Pacific Islander-owned firms, percent, 2007",
                        "SBO415207": "Hispanic-owned firms, percent, 2007",
                        "SBO015207": "Women-owned firms, percent, 2007",
                        "MAN450207": "Manufacturers shipments, 2007 ($1,000)",
                        "WTN220207": "Merchant wholesaler sales, 2007 ($1,000)",
                        "RTN130207": "Retail sales, 2007 ($1,000)",
                        "RTN131207": "Retail sales per capita, 2007",
                        "AFN120207": "Accommodation and food services sales, 2007 ($1,000)",
                        "BPS030214": "Building permits, 2014",
                        "LND110210": "Land area in square miles, 2010",
                        "POP060210": "Population per square mile, 2010"}, inplace=True)

'county_data' (before 'county_facts') columns names codes been renamed after 'county_facts_dictionary' explanations.

### Respublican data

In [34]:
republican_votes_grouped_state = primary_results[primary_results['party'] == 'Republican'].groupby('state').apply(max_votes_candidate).reset_index()
county_data['state'] = county_data['area_name']
republican_votes_x_county_data = pd.merge(republican_votes_grouped_state, county_data, on='state', how='inner')

In [9]:
csv_file_path = "republican_votes_x_county_data.csv"
republican_votes_x_county_data.to_csv(csv_file_path, index=False)

In [35]:
republican_votes_grouped_county = primary_results[primary_results['party'] == 'Republican'].groupby('county').apply(max_votes_candidate).reset_index()
county_data['state'] = county_data['state'].str.replace(" County", "")
republican_votes_grouped_county.rename(columns={'county': 'state'}, inplace=True)
republican_votes_x_county_data_county = pd.merge(republican_votes_grouped_county, county_data, on='state', how='inner')

In [36]:
winner_counts_republicans = republican_votes_x_county_data_county.winner.value_counts()
winner_counts_republicans

winner
Donald Trump    2044
Ted Cruz         638
John Kasich      109
Marco Rubio       49
Ben Carson         1
Name: count, dtype: int64

As we can see Ben Carson Won only in one city so we will drop him as an outlier

In [12]:
republican_votes_x_county_data_county = republican_votes_x_county_data_county[republican_votes_x_county_data_county.winner != 'Ben Carson']

In [13]:
csv_file_path = "republican_votes_x_county_data_county.csv"
republican_votes_x_county_data_county.to_csv(csv_file_path, index=False)

### Democrat data

In [37]:
democrat_votes_grouped_state = primary_results[primary_results['party'] == 'Democrat'].groupby('state').apply(max_votes_candidate).reset_index()
county_data['state'] = county_data['area_name']
democrat_votes_x_county_data = pd.merge(democrat_votes_grouped_state, country_facts, on='state', how='inner')

In [15]:
csv_file_path = "democrat_votes_x_county_data.csv"
democrat_votes_x_county_data.to_csv(csv_file_path, index=False)

In [38]:
democrat_votes_grouped_county = primary_results[primary_results['party'] == 'Democrat'].groupby('county').apply(max_votes_candidate).reset_index()
county_data['state'] = county_data['state'].str.replace(" County", "")
democrat_votes_grouped_county.rename(columns={'county': 'state'}, inplace=True)
democrat_votes_x_county_data_county = pd.merge(democrat_votes_grouped_county, county_data, on='state', how='inner')

In [39]:
winner_counts_democrat = democrat_votes_x_county_data_county.winner.value_counts()
winner_counts_democrat

winner
Hillary Clinton    1668
Bernie Sanders     1224
Name: count, dtype: int64

In [27]:
csv_file_path = "democrat_votes_x_county_data_county.csv"
democrat_votes_x_county_data_county.to_csv(csv_file_path, index=False)